# Amazon Bedrock과 LangChain을 이용한 "비즈니스 데이터 분석을 위한 자연어 기반 BI"

---

노트북 실행을 위해서 아래와 같이 노트북 환경을 Data Science 3.0, Python 3, ml.t3.medium로 지정합니다.
<img src="images/nb-env.png" width="600"/>

---

이 노트북에서는 비즈니스 데이터 분석을 위한 자연어 기반 BI 구현하기 위한 생성형 AI 기반의 Text-to-SQL 기능을 위해 [Amazon Bedrock](https://aws.amazon.com/bedrock/)과 [LangChain](https://www.langchain.com/)을 사용합니다.

Text-to-SQL 기능을 이용한다면 자연어만을 사용해서 구조화된 정보를 보유하고 있는 SQL 데이터베이스에 LLM을 연결하여 특정 비즈니스 데이터에 대한 답을 얻을 수 있습니다. 이때 LLM은 질문에 대한 답을 추출을 위해 데이터베이스에 초점을 맞추게 되며 SQL을 통해 데이터베이스에서 제공하는 정확한 정보를 반환할 수 있습니다. LangChain은 LLM을 다른 데이터소스로 연결할 수 있도록 하는 기능을 제공하여 개발자는 복잡한 애플리케이션을 쉽게 만들 수 있습니다.

<img src="images/sql-agent.png" width="800"/>
Image credits: https://www.langchain.com/

### Text-to-SQL를 이용하여 아래와 같은 사례에 적용할 수 있습니다.
* 데이터베이스 데이터를 기반으로 질문에 답할 수 있는 챗봇 만들기
* 자연어 기반의 복잡한 쿼리 생성
* 비기술 인력 및 이해관계자에 대한 데이터베이스 접근성 확대

이 노트북에서는 사전에 생성된 Amazon Redshift Serverless를 사용하며 데이터베이스와 LLM 연결을 위해 LangChain을 사용합니다.
노트북은 아래와 같은 구성으로 이루어져있습니다.
1. Amazon Redshift Serverless 접속과 데이터 확인
2. LangChain의 SQLDatabaseChain 사용 

Amazon Bedrock과 LangChain을 이용한 Text-to-SQL 구조는 아래 다이어그램을 참조하세요.

<img src="images/rs-architecture.png" width="800"/>

데이터에 대한 질문을 위한 SQL 쿼리를 작성하기 위해서는 원하는 데이터와 조건 등을 지정하는 명령문을 전달합니다. 명령문은 프롬프트와 함께 전달되고 해당되는 데이터베이스에 맞는 SQL 문으로 변환됩니다. SQL문이 실행된 결과 데이터는 다시 LLM에 전달되고 프롬프트에 따라 LLM은 자연어로 결과를 제공해줍니다. 이때 자연어를 이용한 질의를 통해 복잡한 조인까지도 생성할 수 있어 SQL 쿼리를 생성하고 그 결과를 분석하는 것이 정말 쉬워집니다.

---

In [66]:
%pip install --upgrade pip
%pip install -qU redshift_connector sqlalchemy-redshift
# %pip install -qU 'sagemaker>=2.15.0' 'PyAthena==1.10.7' 'SQLAlchemy==1.3.13'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [67]:
import os
import boto3

In [68]:
my_region = os.environ["AWS_DEFAULT_REGION"]   # E.g. "us-east-1"
os.environ["BEDROCK_ENDPOINT_URL"] = f"https://bedrock-runtime.{my_region}.amazonaws.com"  # E.g. "https://..."

STACK_NAME = 'genai-workshop' # Cloudformation의 Stack Name을 입력합니다.

In [69]:
# Redshift Serverless로 연결을 위한 Endpoint를 Cloudformation에서 읽어옵니다.

cf_client = boto3.client('cloudformation')
response = cf_client.describe_stacks()

for output in response["Stacks"]:
    stackName = output["StackName"]
    if stackName.find('Redshift') > 0:
        response = cf_client.describe_stacks(StackName=stackName)
        for output in response["Stacks"][0]["Outputs"]:
            keyName = output["OutputKey"]
            if keyName == "ServerlessWorkgroup":
                WORK_GROUP = output["OutputValue"]
            if keyName == "ServerlessNamespace":
                NAME_SPACE = output["OutputValue"]

rs_client = boto3.client('redshift-serverless')
response = rs_client.get_workgroup(
    workgroupName=WORK_GROUP
)

redshift_endpoint = response['workgroup']['endpoint']['address']

redshift_endpoint

'workshop-workgroup.847828727744.us-east-1.redshift-serverless.amazonaws.com'

# Redshift Serverless 테이블 생성과 데이터 로딩

### 본 노트북에서 사용되는 일반적인 리테일 회사에서 사용될 수 있는 가상의 데이터를 참고했습니다.
- region (P: R_REGIONKEY): 지역 정보; 지역 이름과 설명
- nation (P: N_NATIONKEY): 국가 정보; 국가 이름, 지역 키, 설명
- customer (P: C_CUSTKEY): 고객 정보; 고객의 이름, 주소, 국가 키, 전화번호, 계정 잔액 
- orders (P: O_ORDERKEY): 주문 정보; 주문 상태, 총 가격, 주문일자, 우선 순위; 주문키(distkey), 주문일자(sortkey)
- part (P: P_PARTKEY): 제품 정보; 제품 이름, 제조사, 브랜드, 유형, 크기, 가격
- supplier (P: S_SUPPKEY): 공급업체 정보; 업체 이름, 주소, 국가 키, 전화번호, 계정 잔액
- lineitem (P: L_ORDERKEY): 주문 라인 항목 정보; 주문 제품, 공급업체, 라인 번호, 수량, 확장 가격, 할인, 세금; 주문키(distkey), 수령 일자(sortkey)
- partsupp (P: PS_PARTKEY, PS_SUPPKEY): 제품-공급업체 관계 정보; 제품의 가용 수량, 공급 비용, 설명

## 중요!: 본 실습에 필요한 Redshift Serverless 권한 적용을 위해서는 사용할 IAM Role을 Default Type으로 지정해야합니다.

#### AWS 콘솔에서 Redshift 메뉴로 이동합니다. 그리고 아래와 같이 Redshift Serverless의 Namespace중 `workshop-namespace`를 클릭합니다. 그리고 `Security and encryption`탭에서 실습에 사용할 Role인 `SageMakerUserExecutionRole`을 Default Role Type으로 만들기 위해서 `Manage IAM Role`을 클릭합니다.

<img src="images/namespace.png" width="800"/>

#### `SageMakerUserExecutionRole`을 선택하고 상단의 `Set default`에서 `Make default`를 클릭하여 Role Type을 Default로 만듭니다. 그리고 `반드시` `Save changes`를 클릭합니다.

<img src="images/make-default.png" width="800"/>

In [70]:
import redshift_connector

conn = redshift_connector.connect(
     host=redshift_endpoint,
     database='dev',
     port=5439,
     user="admin",
     password="Admin1234!"
)

conn.autocommit = True
cursor = conn.cursor()

## Redshift Serverless에 테이블 생성

In [71]:
cursor.execute("DROP TABLE IF EXISTS region")

create_statement = """
CREATE TABLE region (
  R_REGIONKEY bigint NOT NULL,
  R_NAME varchar(25),
  R_COMMENT varchar(152))
diststyle all;
"""
cursor.execute(create_statement)


In [72]:
cursor.execute("DROP TABLE IF EXISTS nation")

create_statement = """
CREATE TABLE nation (
  N_NATIONKEY bigint NOT NULL,
  N_NAME varchar(25),
  N_REGIONKEY bigint,
  N_COMMENT varchar(152))
diststyle all;
"""
cursor.execute(create_statement)

In [73]:
cursor.execute("DROP TABLE IF EXISTS customer")

create_statement = """
create table customer (
  C_CUSTKEY bigint NOT NULL,
  C_NAME varchar(25),
  C_ADDRESS varchar(40),
  C_NATIONKEY bigint,
  C_PHONE varchar(15),
  C_ACCTBAL decimal(18,4),
  C_MKTSEGMENT varchar(10),
  C_COMMENT varchar(117))
diststyle all;
"""
cursor.execute(create_statement)

In [74]:
cursor.execute("DROP TABLE IF EXISTS orders")

create_statement = """
create table orders (
  O_ORDERKEY bigint NOT NULL,
  O_CUSTKEY bigint,
  O_ORDERSTATUS varchar(1),
  O_TOTALPRICE decimal(18,4),
  O_ORDERDATE Date,
  O_ORDERPRIORITY varchar(15),
  O_CLERK varchar(15),
  O_SHIPPRIORITY Integer,
  O_COMMENT varchar(79))
distkey (O_ORDERKEY)
sortkey (O_ORDERDATE);
"""
cursor.execute(create_statement)

In [75]:
cursor.execute("DROP TABLE IF EXISTS part")

create_statement = """
create table part (
  P_PARTKEY bigint NOT NULL,
  P_NAME varchar(55),
  P_MFGR  varchar(25),
  P_BRAND varchar(10),
  P_TYPE varchar(25),
  P_SIZE integer,
  P_CONTAINER varchar(10),
  P_RETAILPRICE decimal(18,4),
  P_COMMENT varchar(23))
diststyle all;
"""
cursor.execute(create_statement)

In [76]:
cursor.execute("DROP TABLE IF EXISTS supplier")

create_statement = """
create table supplier (
  S_SUPPKEY bigint NOT NULL,
  S_NAME varchar(25),
  S_ADDRESS varchar(40),
  S_NATIONKEY bigint,
  S_PHONE varchar(15),
  S_ACCTBAL decimal(18,4),
  S_COMMENT varchar(101))
diststyle all;                                                              
"""
cursor.execute(create_statement)

In [77]:
cursor.execute("DROP TABLE IF EXISTS lineitem")

create_statement = """
create table lineitem (
  L_ORDERKEY bigint NOT NULL,
  L_PARTKEY bigint,
  L_SUPPKEY bigint,
  L_LINENUMBER integer NOT NULL,
  L_QUANTITY decimal(18,4),
  L_EXTENDEDPRICE decimal(18,4),
  L_DISCOUNT decimal(18,4),
  L_TAX decimal(18,4),
  L_RETURNFLAG varchar(1),
  L_LINESTATUS varchar(1),
  L_SHIPDATE date,
  L_COMMITDATE date,
  L_RECEIPTDATE date,
  L_SHIPINSTRUCT varchar(25),
  L_SHIPMODE varchar(10),
  L_COMMENT varchar(44))
distkey (L_ORDERKEY)
sortkey (L_RECEIPTDATE);
"""
cursor.execute(create_statement)

In [78]:
cursor.execute("DROP TABLE IF EXISTS partsupp")

create_statement = """
create table partsupp (
  PS_PARTKEY bigint NOT NULL,
  PS_SUPPKEY bigint NOT NULL,
  PS_AVAILQTY integer,
  PS_SUPPLYCOST decimal(18,4),
  PS_COMMENT varchar(199))
diststyle even;
"""
cursor.execute(create_statement)

## Redshift Serverless에 데이터 로딩

### Note: S3부터 데이터 Copy를 위해서는 반드시 앞에서 설명한 Set Default IAM Role in Redshift Serverless을 수행해야합니다.

In [79]:
load_data_statement = """
COPY region FROM 's3://redshift-immersionday-labs/data/region/region.tbl.lzo'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;
"""
cursor.execute(load_data_statement)

In [80]:
load_data_statement = """
COPY nation FROM 's3://redshift-immersionday-labs/data/nation/nation.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;
"""
cursor.execute(load_data_statement)

In [81]:
load_data_statement = """
copy customer from 's3://redshift-immersionday-labs/data/customer/customer.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;
"""
cursor.execute(load_data_statement)

In [82]:
load_data_statement = """
copy orders from 's3://redshift-immersionday-labs/data/orders/orders.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;
"""
cursor.execute(load_data_statement)

In [83]:
load_data_statement = """
copy part from 's3://redshift-immersionday-labs/data/part/part.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;
"""
cursor.execute(load_data_statement)

In [84]:
load_data_statement = """
copy supplier from 's3://redshift-immersionday-labs/data/supplier/supplier.json' manifest
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;
"""
cursor.execute(load_data_statement)

In [85]:
load_data_statement = """
copy lineitem from 's3://redshift-immersionday-labs/data/lineitem-part/'
iam_role default
region 'us-west-2' gzip delimiter '|' COMPUPDATE PRESET;
"""
cursor.execute(load_data_statement)

In [86]:
load_data_statement = """
copy partsupp from 's3://redshift-immersionday-labs/data/partsupp/partsupp.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;
"""
cursor.execute(load_data_statement)

Redshift Serverless 데이터 로딩 확인

In [87]:
%pip install --quiet ipython-sql
%pip install --quiet --upgrade SQLAlchemy==1.*
%pip install -qU psycopg2-binary

import pandas as pd
import sqlalchemy

username = "admin" 
password = "Admin1234!" 
host = redshift_endpoint 
port = "5439"
mydatabase = "dev"

redshift_uri = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{mydatabase}"

engine = sqlalchemy.create_engine(redshift_uri)
df = pd.read_sql('select * from orders limit 10', engine)
df.head()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy-redshift 0.8.14 requires SQLAlchemy<2.0.0,>=0.9.2, but you have sqlalchemy 2.0.23 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.50 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,28009541,13096240,F,118898.15,1992-01-01,3-MEDIUM,Clerk#000014614,0,"beans. express, furious accounts wake slyly a..."
1,216029091,2951965,F,49717.77,1992-01-01,4-NOT SPECIFIED,Clerk#000094912,0,ounts haggle blithely about the multipliers. e...
2,232075587,9762419,F,202931.54,1992-01-01,5-LOW,Clerk#000087238,0,eposits. slyly ironic ideas use quickly. final
3,212259525,12576386,F,192933.33,1992-01-01,2-HIGH,Clerk#000003049,0,"ost final, express deposits. furiously regular..."
4,292068259,4419706,F,79728.74,1992-01-01,5-LOW,Clerk#000071174,0,y regular foxes are. quietly ironic notornis n...


# LangChain SQLDatabase 설정

In [88]:
%pip install -qU langchain langchain-experimental

Note: you may need to restart the kernel to use updated packages.


In [89]:
import os
import sys
import boto3

from langchain.llms.bedrock import Bedrock

BEDROCK_ENDPOINT_URL = os.environ["BEDROCK_ENDPOINT_URL"] 

session = boto3.Session(
    profile_name=os.environ.get("AWS_PROFILE")
) 

bedrock = session.client(
    service_name='bedrock-runtime', 
    region_name=os.environ.get("AWS_DEFAULT_REGION"),
    endpoint_url=BEDROCK_ENDPOINT_URL
) 

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=bedrock, model_kwargs={'max_tokens_to_sample':1000, 'temperature':0})

In [90]:
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri(redshift_uri)

In [91]:
print(db.table_info)


CREATE TABLE customer (
	c_custkey BIGINT NOT NULL, 
	c_name VARCHAR(25), 
	c_address VARCHAR(40), 
	c_nationkey BIGINT, 
	c_phone VARCHAR(15), 
	c_acctbal NUMERIC(18, 4), 
	c_mktsegment VARCHAR(10), 
	c_comment VARCHAR(117)
)

/*
3 rows from customer table:
c_custkey	c_name	c_address	c_nationkey	c_phone	c_acctbal	c_mktsegment	c_comment
5300001	Customer#005300001	QeruwD,h1mgbqsFyF,taVoTUa7Sg7BaqSsY	11	21-585-117-8821	7943.2300	BUILDING	uffily. unusual requests use carefully alongside of the blithely unu
5300002	Customer#005300002	4eifAz5FUgq	8	18-761-669-4319	3858.0900	AUTOMOBILE	nic requests. bold foxes haggle slyly. furiously even pains wake fur
5300003	Customer#005300003	peXDPDH vGki2uL2vkFvA0uEy2QB7TLc	13	23-782-864-8661	6953.8300	FURNITURE	ular requests alongside of the sauternes are furiously against the final, ironic foxes. carefully sp
*/


CREATE TABLE lineitem (
	l_orderkey BIGINT NOT NULL, 
	l_partkey BIGINT, 
	l_suppkey BIGINT, 
	l_linenumber INTEGER NOT NULL, 
	l_quantity

## 프롬프트에 따라 사용자 질문을 기반으로 SQL 쿼리를 작성하고 실행하는 체인을 생성하는 코드 블록

In [92]:
PROMPT = """ 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: {question}

Assistant:"""

db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True) # verbose=True로 실행되는 SQL을 상세하게 보여줍니다.

question = "공급업체의 평균 계정 잔액은 어떻게 돼?"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 공급업체의 평균 계정 잔액은 어떻게 돼?

Assistant:
SQLQuery:SELECT AVG(s_acctbal) AS avg_acctbal
FROM supplier;
SQLResult: [(Decimal('4500.0568'),)]
Answer:공급업체의 평균 계정 잔액은 4500.0568입니다.
> Finished chain.


'공급업체의 평균 계정 잔액은 4500.0568입니다.'

#### 쿼리 실행없이 SQL문 확인

In [93]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question":"공급업체의 평균 계정 잔액은 어떻게 돼?"})

print(response)

공급업체의 평균 계정 잔액을 구하기 위해 다음과 같은 SQL 쿼리를 사용할 수 있습니다:

```sql
SELECT AVG("s_acctbal") AS "avg_acctbal"
FROM supplier
```

이 쿼리는 supplier 테이블에서 s_acctbal 컬럼의 평균값을 구합니다.

SQLQuery:
SELECT AVG("s_acctbal") AS "avg_acctbal"
FROM supplier


#### 데이터베이스에서 SQL문 실행

In [94]:
sql_statement = "SELECT AVG(s_acctbal) AS avg_acct_balance FROM supplier"
df = pd.read_sql(sql_statement, engine)
df.head()

,avg_acct_balance
0,4500.0568


In [95]:
question = "어떤 국가의 고객이 가장 많이 주문했나요?"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 어떤 국가의 고객이 가장 많이 주문했나요?

Assistant:
SQLQuery:SELECT n.n_name, COUNT(o.o_orderkey) AS order_count
FROM nation n
JOIN customer c ON n.n_nationkey = c.c_nationkey 
JOIN orders o ON c.c_custkey = o.o_custkey
GROUP BY n.n_name
ORDER BY order_count DESC
LIMIT 1;
SQLResult: [('INDONESIA', 3049795)]
Answer:인도네시아
> Finished chain.


'인도네시아'

In [96]:
question = "주문 수가 가장 많은 국가는 어디인가요?"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 주문 수가 가장 많은 국가는 어디인가요?

Assistant:
SQLQuery:SELECT n_name, COUNT(*) AS order_count
FROM nation n
JOIN customer c ON n.n_nationkey = c.c_nationkey
JOIN orders o ON c.c_custkey = o.o_custkey
GROUP BY n_name
ORDER BY order_count DESC
LIMIT 1
SQLResult: [('INDONESIA', 3049795)]
Answer:인도네시아
> Finished chain.


'인도네시아'

In [97]:
question = "고객의 평균 계정 잔액은 어떻게 되나요?"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 고객의 평균 계정 잔액은 어떻게 되나요?

Assistant:
SQLQuery:SELECT AVG(c_acctbal) AS avg_acct_bal
FROM customer;
SQLResult: [(Decimal('4498.5519'),)]
Answer:고객의 평균 계정 잔액은 4498.5519입니다.
> Finished chain.


'고객의 평균 계정 잔액은 4498.5519입니다.'

In [98]:
question = "어떤 고객이 가장 많은 주문을 했나요?"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 어떤 고객이 가장 많은 주문을 했나요?

Assistant:
SQLQuery:SELECT c_name, COUNT(*) AS num_orders
FROM orders o JOIN customer c ON o.o_custkey = c.c_custkey
GROUP BY c_name
ORDER BY num_orders DESC
LIMIT 1;
SQLResult: [('Customer#012340771', 29)]
Answer:Customer#012340771
> Finished chain.


'Customer#012340771'

In [99]:
sql_statement = "select * from customer  where c_name like 'Customer#009100001'"
df = pd.read_sql(sql_statement, engine)
df.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,9100001,Customer#009100001,"IIGY3g9IyWC,nYmA60Df0T",0,10-520-278-2536,8143.07,HOUSEHOLD,"to the express requests wake regular, f"


In [100]:
query_statement = """
SELECT c_name 
FROM customer JOIN orders ON o_custkey = c_custkey
GROUP BY c_name
ORDER BY COUNT(*) DESC
LIMIT 1
"""
df = pd.read_sql(query_statement, engine)
df.head()

,c_name
0,Customer#012340771


In [101]:
question = "마케팅 세그먼트별 고객이 주문하는 양은 어떻게 다른가요? "

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 마케팅 세그먼트별 고객이 주문하는 양은 어떻게 다른가요? 

Assistant:
SQLQuery:SELECT c_mktsegment, AVG(l_quantity) AS avg_quantity
FROM customer c JOIN orders o ON c.c_custkey = o.o_custkey
             JOIN lineitem l ON o.o_orderkey = l.l_orderkey
GROUP BY c_mktsegment
ORDER BY avg_quantity DESC;
SQLResult: [('BUILDING', Decimal('25.5031')), ('AUTOMOBILE', Decimal('25.4996')), ('FURNITURE', Decimal('25.4990')), ('HOUSEHOLD', Decimal('25.4986')), ('MACHINERY', Decimal('25.4983'))]
Answer:마케팅 세그먼트별로 BUILDING 세그먼트의 고객이 가장 많은 양을 주문하고, AUTOMOBILE, FURNITURE, HOUSEHOLD, MACHINERY 순으로 주문량이 많습니다.
> Finished chain.


'마케팅 세그먼트별로 BUILDING 세그먼트의 고객이 가장 많은 양을 주문하고, AUTOMOBILE, FURNITURE, HOUSEHOLD, MACHINERY 순으로 주문량이 많습니다.'

In [102]:
question = "마케팅 세그먼트별 고객이 주문하는 양을 알려주세요"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 마케팅 세그먼트별 고객이 주문하는 양을 알려주세요

Assistant:
SQLQuery:SELECT c_mktsegment, SUM(l_quantity) AS total_quantity
FROM customer c JOIN orders o ON c.c_custkey = o.o_custkey
             JOIN lineitem l ON o.o_orderkey = l.l_orderkey
GROUP BY c_mktsegment
ORDER BY total_quantity DESC;
SQLResult: [('FURNITURE', Decimal('1546275558.0000')), ('AUTOMOBILE', Decimal('1545790383.0000')), ('HOUSEHOLD', Decimal('1545155899.0000')), ('BUILDING', Decimal('1544821487.0000')), ('MACHINERY', Decimal('1544594058.0000'))]
Answer:마케팅 세그먼트별로 FURNITURE 세그먼트의 고객이 가장 많은 양을 주문하고, BUILDING 세그먼트의 고객이 가장 적은 양을 주문합니다.
> Finished chain.


'마케팅 세그먼트별로 FURNITURE 세그먼트의 고객이 가장 많은 양을 주문하고, BUILDING 세그먼트의 고객이 가장 적은 양을 주문합니다.'

In [103]:
question = "주문 상태별 주문 수와 총 매출은 어떻게 되나요?"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 주문 상태별 주문 수와 총 매출은 어떻게 되나요?

Assistant:
SQLQuery:SELECT o_orderstatus, COUNT(*) AS order_count, SUM(o_totalprice) AS total_revenue
FROM orders
GROUP BY o_orderstatus
SQLResult: [('O', 37030175, Decimal('5563129791682.3000')), ('P', 1945703, Decimal('359572088111.3400')), ('F', 37024122, Decimal('5562126040598.7000'))]
Answer:O 상태 주문 수는 37030175개이고 총 매출은 5563129791682.3000입니다. P 상태 주문 수는 1945703개이고 총 매출은 359572088111.3400입니다. F 상태 주문 수는 37024122개이고 총 매출은 5562126040598.7000입니다.
> Finished chain.


'O 상태 주문 수는 37030175개이고 총 매출은 5563129791682.3000입니다. P 상태 주문 수는 1945703개이고 총 매출은 359572088111.3400입니다. F 상태 주문 수는 37024122개이고 총 매출은 5562126040598.7000입니다.'

In [104]:
query_statement = """
SELECT o_orderstatus, COUNT(*) AS order_count, SUM(o_totalprice) AS total_price
FROM orders 
GROUP BY o_orderstatus;
"""
df = pd.read_sql(query_statement, engine)
df.head()

,o_orderstatus,order_count,total_price
0,O,37030175,5.563130e+12
1,F,37024122,5.562126e+12
2,P,1945703,3.595721e+11


In [105]:
question = "주문 금액이 가장 큰 주문 금액과 주문 정보 알려주세요"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 주문 금액이 가장 큰 주문 금액과 주문 정보 알려주세요

Assistant:
SQLQuery:SELECT o_totalprice, o_orderkey, o_orderdate, o_shippriority 
FROM orders
ORDER BY o_totalprice DESC
LIMIT 1
SQLResult: [(Decimal('571417.4800'), 84927619, datetime.date(1997, 6, 29), 0)]
Answer:주문 금액이 가장 큰 주문은 571417.4800이며, 주문 번호는 84927619, 주문일자는 1997년 6월 29일, 배송 우선순위는 0입니다.
> Finished chain.


'주문 금액이 가장 큰 주문은 571417.4800이며, 주문 번호는 84927619, 주문일자는 1997년 6월 29일, 배송 우선순위는 0입니다.'

In [106]:
query_statement = """
SELECT o_orderkey, o_totalprice, o_orderdate, o_orderpriority 
FROM orders
ORDER BY o_totalprice DESC
LIMIT 1;
"""
df = pd.read_sql(query_statement, engine)
df.head()

,o_orderkey,o_totalprice,o_orderdate,o_orderpriority
0,84927619,571417.48,1997-06-29,1-URGENT


In [107]:
question = "주문일자에 따른 주문량 및 매출의 월별 추이는 어떻게 되나요?"

db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 주문일자에 따른 주문량 및 매출의 월별 추이는 어떻게 되나요?

Assistant:
SQLQuery:SELECT
  EXTRACT(MONTH FROM o_orderdate) AS month,
  SUM(l_quantity) AS total_quantity,
  SUM(l_extendedprice) AS total_sales
FROM orders o
JOIN lineitem l ON o.o_orderkey = l.l_orderkey
GROUP BY month
ORDER BY month;
SQLResult: [(1, Decimal('697290985.0000'), Decimal('1045595065791.4900')), (2, Decimal('636058954.0000'), Decimal('953756754701.5800')), (3, Decimal('696516117.0000'), Decimal('1044454247917.8400')), (4, Decimal('674418282.0000'), Decimal('1011286485722.5400')), (5, Decimal('696884101.0000'), Decimal('1044993215415.0000')), (6, Decimal('674743829.0000'), Decimal('1011731003376.0400')), (7, Decimal('696958547.0000'), Decimal('1045063289503.0800')), (8, Decimal('603982139.0000'), Decimal('905688836546

'월별 주문량은 5-7월에 가장 많고, 매출도 5-7월에 가장 높습니다.'

In [108]:
query_statement = """
SELECT
  DATE_TRUNC('month', o_orderdate) AS month,
  SUM(l_quantity) AS total_qty,
  SUM(l_extendedprice * (1 - l_discount)) AS total_amount
FROM orders o
JOIN lineitem l ON o.o_orderkey = l.l_orderkey  
GROUP BY 1
ORDER BY 1;
"""
df = pd.read_sql(query_statement, engine)
df.head(100)

,month,total_qty,total_amount
0,1992-01-01,99663205.0,1.419620e+11
1,1992-02-01,93107547.0,1.326408e+11
2,1992-03-01,99238813.0,1.413739e+11
3,1992-04-01,96293391.0,1.371863e+11
4,1992-05-01,99497085.0,1.417110e+11
...,...,...,...
75,1998-04-01,96304953.0,1.371860e+11
76,1998-05-01,99702770.0,1.420418e+11
77,1998-06-01,96278050.0,1.371642e+11
78,1998-07-01,99558670.0,1.418191e+11


In [109]:
db_chain.run(PROMPT.format(question="주문일자에 따른 주문량 및 매출의 월별 추이는 어떻게 되나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 주문일자에 따른 주문량 및 매출의 월별 추이는 어떻게 되나요?

Assistant:
SQLQuery:SELECT
  EXTRACT(MONTH FROM o_orderdate) AS month,
  SUM(l_quantity) AS total_quantity,
  SUM(l_extendedprice) AS total_sales
FROM orders o
JOIN lineitem l ON o.o_orderkey = l.l_orderkey
GROUP BY month
ORDER BY month;
SQLResult: [(1, Decimal('697290985.0000'), Decimal('1045595065791.4900')), (2, Decimal('636058954.0000'), Decimal('953756754701.5800')), (3, Decimal('696516117.0000'), Decimal('1044454247917.8400')), (4, Decimal('674418282.0000'), Decimal('1011286485722.5400')), (5, Decimal('696884101.0000'), Decimal('1044993215415.0000')), (6, Decimal('674743829.0000'), Decimal('1011731003376.0400')), (7, Decimal('696958547.0000'), Decimal('1045063289503.0800')), (8, Decimal('603982139.0000'), Decimal('905688836546

'월별 주문량은 5-7월에 가장 많고, 매출도 5-7월에 가장 높습니다.'

In [110]:
query_statement = """
SELECT EXTRACT(MONTH FROM o_orderdate) AS order_month, 
       SUM(l_extendedprice * (1 - l_discount)) AS revenue,
       COUNT(o_orderkey) AS order_count
FROM orders o
JOIN lineitem l ON o.o_orderkey = l.l_orderkey
GROUP BY order_month
ORDER BY order_month;
"""
df = pd.read_sql(query_statement, engine)
df.head(12)

,order_month,revenue,order_count
0,1,9.933161e+11,27344530
1,2,9.060703e+11,24944177
2,3,9.922287e+11,27314034
3,4,9.607262e+11,26450594
4,5,9.927521e+11,27330612
5,6,9.611474e+11,26460707
6,7,9.928100e+11,27330157
7,8,8.604046e+11,23687868
8,9,8.226558e+11,22650852
9,10,8.506984e+11,23415496


In [111]:
db_chain.run(PROMPT.format(question="어떤 주문 우선 순위가 가장 일반적인가요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 어떤 주문 우선 순위가 가장 일반적인가요?

Assistant:
SQLQuery:SELECT o_orderpriority, COUNT(*) AS order_count
FROM orders 
GROUP BY o_orderpriority
ORDER BY order_count DESC
LIMIT 1;
SQLResult: [('4-NOT SPECIFIED', 15202181)]
Answer:4-NOT SPECIFIED
> Finished chain.


'4-NOT SPECIFIED'

In [112]:
db_chain.run(PROMPT.format(question="주문 수가 가장 많은 점원은 어떻게 되나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 주문 수가 가장 많은 점원은 어떻게 되나요?

Assistant:
SQLQuery:SELECT o_clerk, COUNT(*) AS order_count
FROM orders
GROUP BY o_clerk
ORDER BY order_count DESC
LIMIT 1;
SQLResult: [('Clerk#000049229', 872)]
Answer:Clerk#000049229
> Finished chain.


'Clerk#000049229'

In [113]:
db_chain.run(PROMPT.format(question="어떤 제품 카테고리가 가장 많이 주문되었나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 어떤 제품 카테고리가 가장 많이 주문되었나요?

Assistant:
SQLQuery:SELECT p_type, COUNT(*) AS num_orders
FROM lineitem l JOIN part p ON l.l_partkey = p.p_partkey
GROUP BY p_type
ORDER BY num_orders DESC
LIMIT 1;
SQLResult: [('SMALL ANODIZED TIN', 2035649)]
Answer:SMALL ANODIZED TIN
> Finished chain.


'SMALL ANODIZED TIN'

In [114]:
db_chain.run(PROMPT.format(question="가장 비싼 제품은 무엇이며, 어떤 주문에서 판매되었나요? "))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 가장 비싼 제품은 무엇이며, 어떤 주문에서 판매되었나요? 

Assistant:
SQLQuery:SELECT p.p_name, o.o_orderkey
FROM part p
JOIN lineitem l ON p.p_partkey = l.l_partkey
JOIN orders o ON l.l_orderkey = o.o_orderkey
ORDER BY p.p_retailprice DESC
LIMIT 1;
SQLResult: [('goldenrod drab brown salmon mint', 35754244)]
Answer:가장 비싼 제품은 goldenrod drab brown salmon mint이며, 35754244번 주문에서 판매되었습니다.
> Finished chain.


'가장 비싼 제품은 goldenrod drab brown salmon mint이며, 35754244번 주문에서 판매되었습니다.'

In [115]:
db_chain.run(PROMPT.format(question="제품 크기에 따른 주문량은 어떻게 다른가요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 제품 크기에 따른 주문량은 어떻게 다른가요?

Assistant:
SQLQuery:SELECT p_size, COUNT(*) AS order_count
FROM lineitem l JOIN part p ON l.l_partkey = p.p_partkey
GROUP BY p_size
ORDER BY p_size;
SQLResult: [(1, 6062773), (2, 6043286), (3, 6065119), (4, 6061562), (5, 6063176), (6, 6049574), (7, 6069786), (8, 6070453), (9, 6072133), (10, 6068272), (11, 6063010), (12, 6057022), (13, 6072878), (14, 6055116), (15, 6059402), (16, 6061363), (17, 6048716), (18, 6050287), (19, 6071033), (20, 6063818), (21, 6045115), (22, 6070893), (23, 6073164), (24, 6063351), (25, 6065164), (26, 6049634), (27, 6060175), (28, 6064904), (29, 6066592), (30, 6042047), (31, 6057695), (32, 6052386), (33, 6059987), (34, 6054078), (35, 6075823), (36, 6059881), (37, 6050385), (38, 6071890), (39, 6062866), (40, 6055941), 

'제품 크기가 작을수록 주문량이 많고, 제품 크기가 커질수록 주문량이 감소하는 경향이 있습니다. 1~15사이즈의 제품이 가장 많이 주문된 것으로 보입니다.'

In [116]:
db_chain.run(PROMPT.format(question="가장 많이 재주문된 제품은?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 가장 많이 재주문된 제품은?

Assistant:
SQLQuery:SELECT p_name
FROM part
WHERE p_partkey = (
  SELECT l_partkey
  FROM lineitem
  GROUP BY l_partkey
  ORDER BY COUNT(*) DESC
  LIMIT 1
);
SQLResult: [('dim spring sandy brown slate',)]
Answer:dim spring sandy brown slate
> Finished chain.


'dim spring sandy brown slate'

In [117]:
db_chain.run(PROMPT.format(question="어떤 공급업체가 가장 많은 제품을 공급하였나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 어떤 공급업체가 가장 많은 제품을 공급하였나요?

Assistant:
SQLQuery:SELECT s_name, COUNT(*) AS num_products
FROM supplier s JOIN partsupp ps ON s.s_suppkey = ps.ps_suppkey
GROUP BY s_name
ORDER BY num_products DESC
LIMIT 1;
SQLResult: [('Supplier#000800018', 160)]
Answer:Supplier#000800018
> Finished chain.


'Supplier#000800018'

In [118]:
db_chain.run(PROMPT.format(question="공급업체의 평균 계정 잔액은 어떻게 되나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 공급업체의 평균 계정 잔액은 어떻게 되나요?

Assistant:
SQLQuery:SELECT AVG(s_acctbal) AS avg_acctbal
FROM supplier;
SQLResult: [(Decimal('4500.0568'),)]
Answer:공급업체의 평균 계정 잔액은 4500.0568입니다.
> Finished chain.


'공급업체의 평균 계정 잔액은 4500.0568입니다.'

In [119]:
db_chain.run(PROMPT.format(question="공급업체의 평균 공급 비용은 어떻게 되나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 공급업체의 평균 공급 비용은 어떻게 되나요?

Assistant:
SQLQuery:SELECT AVG(ps_supplycost) AS avg_supplycost
FROM partsupp;
SQLResult: [(Decimal('500.4743'),)]
Answer:공급업체의 평균 공급 비용은 500.4743입니다.
> Finished chain.


'공급업체의 평균 공급 비용은 500.4743입니다.'

In [120]:
db_chain.run(PROMPT.format(question="어떤 주문 라인 항목이 가장 높은 할인을 받았나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 어떤 주문 라인 항목이 가장 높은 할인을 받았나요?

Assistant:
SQLQuery:SELECT l_orderkey, l_discount 
FROM lineitem
ORDER BY l_discount DESC
LIMIT 1;
SQLResult: [(241309445, Decimal('0.1000'))]
Answer:주문 라인 항목 241309445이 가장 높은 할인율 0.1000을 받았습니다.
> Finished chain.


'주문 라인 항목 241309445이 가장 높은 할인율 0.1000을 받았습니다.'

In [121]:
db_chain.run(PROMPT.format(question="주문 라인 항목의 평균 수량은 어떻게 되나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 주문 라인 항목의 평균 수량은 어떻게 되나요?

Assistant:
SQLQuery:SELECT AVG(l_quantity) AS avg_quantity
FROM lineitem;
SQLResult: [(Decimal('25.4997'),)]
Answer:주문 라인 항목의 평균 수량은 25.4997입니다.
> Finished chain.


'주문 라인 항목의 평균 수량은 25.4997입니다.'

In [122]:
db_chain.run(PROMPT.format(question="어떤 배송 방법이 가장 많이 사용되었나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query execution and return only the answer.  
The query: 어떤 배송 방법이 가장 많이 사용되었나요?

Assistant:
SQLQuery:SELECT l_shipmode, COUNT(*) AS num_orders
FROM lineitem 
GROUP BY l_shipmode
ORDER BY num_orders DESC
LIMIT 1;
SQLResult: [('RAIL', 43294267)]
Answer:RAIL
> Finished chain.


'RAIL'

In [123]:
PROMPT = """ 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query and return the detailed answer in Korean.  
The query: {question}

Assistant:"""

In [124]:
db_chain.run(PROMPT.format(question="어떤 배송 방법이 가장 많이 사용되었나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query and return the detailed answer in Korean.  
The query: 어떤 배송 방법이 가장 많이 사용되었나요?

Assistant:
SQLQuery:SELECT l_shipmode, COUNT(*) AS count
FROM lineitem 
GROUP BY l_shipmode
ORDER BY count DESC
LIMIT 1;
SQLResult: [('RAIL', 43294267)]
Answer:레일(RAIL) 배송 방법이 가장 많이 사용되었습니다.
> Finished chain.


'레일(RAIL) 배송 방법이 가장 많이 사용되었습니다.'

In [125]:
PROMPT = """ 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query and return the detailed answer as much as possible based on the result.  
The query: {question}

Assistant:"""

db_chain.run(PROMPT.format(question="어떤 배송 방법이 가장 많이 사용되었나요?"))



> Entering new SQLDatabaseChain chain...
 
Human: Given an input question, create a syntactically correct postgresql query to run,
then look at the results of the query and return the detailed answer as much as possible based on the result.  
The query: 어떤 배송 방법이 가장 많이 사용되었나요?

Assistant:
SQLQuery:SELECT l_shipmode, COUNT(*) AS num_orders
FROM lineitem 
GROUP BY l_shipmode
ORDER BY num_orders DESC
LIMIT 1;
SQLResult: [('RAIL', 43294267)]
Answer:'RAIL' 배송 방법이 가장 많이 사용되었습니다.
> Finished chain.


"'RAIL' 배송 방법이 가장 많이 사용되었습니다."

***
#Clean Up
***
## Option 1

### 아래와 같이 Cloudformation에서 genai-workshop-Redshift-*** 스택을 삭제합니다. 

In [ ]:
cf_client = boto3.client('cloudformation')
response = cf_client.describe_stacks()

for output in response["Stacks"]:
    stackName = output["StackName"]
    if stackName.find('Redshift') > 0:
        response = cf_client.delete_stack(StackName=stackName)

response

***
## Option 2

### 아래와 같이 Cloudformation에서 상단의 "View nested"를 선택하고 삭제할 genai-workshop-Redshift-*** 스택을 선택합니다. 그리고 우측 스택 상세화면에서 "Delete"를 선택합니다.   
<img src="images/cf-d-1.png" width="600"/>

### 아래와 같이 "De "Delete"를 선택합니다.   
<img src="images/cf-d-2.png" width="600"/>